In [1]:
from importlib import reload
import sys
sys.path.insert(0, '../')

# sys.path.remove("/home/users/yixiuz/.local/lib/python3.9/site-packages")
sys.path.append("/home/groups/swl1/yixiuz/torch_fid/lib/python3.9/site-packages")
sys.path.append("/home/groups/swl1/yixiuz/torch_fid/bin")

In [2]:
import matplotlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import ml_collections

# import config.eval.genes_base as eval_config
import config.eval.genes_hollow as eval_config

import lib.utils.bookkeeping as bookkeeping
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

import lib.utils.utils as utils
import lib.models.models as models
import lib.models.model_utils as model_utils
import lib.datasets.datasets as datasets
import lib.datasets.dataset_utils as dataset_utils
import lib.sampling.sampling as sampling
import lib.sampling.sampling_utils as sampling_utils

%matplotlib inline

eval_cfg = eval_config.get_config()
train_cfg = bookkeeping.load_ml_collections(Path(eval_cfg.train_config_path))

for item in eval_cfg.train_config_overrides:
    utils.set_in_nested_dict(train_cfg, item[0], item[1])

S = train_cfg.data.S
device = torch.device(eval_cfg.device)
# device = torch.device('cpu')

model = model_utils.create_model(train_cfg, device)

loaded_state = torch.load(Path(eval_cfg.checkpoint_path),
    map_location=device)

modified_model_state = utils.remove_module_from_keys(loaded_state['model'])
model.load_state_dict(modified_model_state)

model.eval()

dataset = dataset_utils.get_dataset(eval_cfg, device)
data = dataset.data

2024-05-21 14:45:12.384805: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 14:45:12.494423: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /share/software/user/open/cudnn/8.9.0.131/lib:/usr/lib64/nvidia:/share/software/user/open/cuda/12.2.0/targets/x86_64-linux/lib:/share/software/user/open/cuda/12.2.0/lib64:/share/software/user/open/cuda/12.2.0/nvvm/lib64:/share/software/user/open/cuda/12.2.0/extras/Debugger/lib64:/share/software/user/open/cuda/12.2.0/extras/CUPTI/lib64:/share/software/user/open/python/3.9.0/lib:/share/software/u

In [5]:
from eval import count_mistakes, get_dist, hellinger, eval_mse_stats, save_results, eval_dct_mse_stats
from tqdm import tqdm

In [6]:
def run_experiments(num_repeats, sample_size, results_file,
                   method, g_steps, tau_steps, 
                    c_steps, c_stepsize, corrector):
    
    tqdm._instances.clear()

    # Tau-leaping only
    eval_cfg.sampler.num_steps = tau_steps
    eval_cfg.sampler.updates_per_eval = g_steps
    
    # Generic corrector fields
    eval_cfg.sampler.num_corrector_steps = c_steps
    eval_cfg.sampler.corrector_entry_time = 0.9
    eval_cfg.sampler.corrector_step_size_multiplier = c_stepsize
    eval_cfg.sampler.balancing_function = corrector

    if method == "gillespies":
        eval_cfg.sampler.name = "PCMultiGillespies"
    elif method == "tauleaping":
        eval_cfg.sampler.name = "PCTauLeapingAbsorbingInformed"
    else:
        assert(False)

    sampler = sampling_utils.get_sampler(eval_cfg)

    results = []

    for _ in range(num_repeats):

        samples, out = sampler.sample(model, sample_size, 0)

        samples = np.clip(samples, 0, S-2)
        time = eval_mse_stats(dataset, samples, S)
        freq = eval_dct_mse_stats(dataset, samples, S)
        
        D = eval_cfg.data.shape[0]
        if method == "gillespies":
            nfe = D / g_steps
        elif method == "tauleaping":
            nfe = eval_cfg.sampler.num_steps
        nfe += nfe * eval_cfg.sampler.corrector_entry_time * eval_cfg.sampler.num_corrector_steps

        new_result = {
                    'method': method,
                    'g_steps': 0 if method != "gillespies" else g_steps,
                    'tau_steps': 0 if method != "tauleaping" else eval_cfg.sampler.num_steps,
                    'use_corrector': eval_cfg.sampler.corrector_entry_time > 0 
                                 and eval_cfg.sampler.num_corrector_steps > 0,
                    'corrector': eval_cfg.sampler.balancing_function,
                    'c_stepsize': eval_cfg.sampler.corrector_step_size_multiplier,
                    'c_steps': eval_cfg.sampler.num_corrector_steps,
                    'nfe': nfe,
#                     'h_dist': h_dist,
#                     'error_rate': error_rate,
                    'time_mean': time["mean"],
                    'time_var' : time["variance"],
                    'time_skew': time["skewness"],
                    'time_kurt': time["kurtosis"],
                    'freq_mean': freq["mean"],
                    'freq_var' : freq["variance"],
                    'freq_skew': freq["skewness"],
                    'freq_kurt': freq["kurtosis"],
                }
        print(new_result)
        results.append(new_result)

    save_results(results, results_file)

In [7]:
results_file = 'sequence_results_test_1.csv'

num_repeats = 10
sample_size = 100

methods = ["tauleaping", 
           "gillespies"]

updates_per_evals = [3, 2, 1]
num_sample_steps = [50, 100, 200]

correctors = ["mpf", "barker", "birthdeath"]
corrector_stepsizes = [2.0, 1.0, 0.1]
corrector_steps = 2

for method in methods:
    for i in range(len(correctors)):
        corrector = correctors[i]
        c_stepsize = corrector_stepsizes[i]
        g_steps = updates_per_evals[0]
        tau_steps = num_sample_steps[0]
        c_steps = corrector_steps
        run_experiments(num_repeats=num_repeats, 
                        sample_size=sample_size, 
                        results_file=results_file,
                        method=method, 
                        g_steps=g_steps, 
                        tau_steps=tau_steps, 
                        c_steps=c_steps, 
                        c_stepsize=c_stepsize, 
                        corrector=corrector)

49it [00:02, 24.15it/s]


{'method': 'tauleaping', 'g_steps': 0, 'tau_steps': 50, 'use_corrector': True, 'corrector': 'mpf', 'c_stepsize': 2.0, 'c_steps': 2, 'nfe': 140.0, 'time_mean': 0.07527652534613645, 'time_var': 0.01155189256843057, 'time_skew': 1.3726797759049172, 'time_kurt': 7.610480301989932, 'freq_mean': 0.0752765316815486, 'freq_var': 0.14947265788890995, 'freq_skew': 1.1974703857823745, 'freq_kurt': 17.572843050100825}


49it [00:01, 33.04it/s]


{'method': 'tauleaping', 'g_steps': 0, 'tau_steps': 50, 'use_corrector': True, 'corrector': 'mpf', 'c_stepsize': 2.0, 'c_steps': 2, 'nfe': 140.0, 'time_mean': 0.0745560407072662, 'time_var': 0.015302085577777874, 'time_skew': 1.4548178426177971, 'time_kurt': 6.259159745058786, 'freq_mean': 0.07455604587075632, 'freq_var': 0.17974159320253927, 'freq_skew': 0.9736287901935945, 'freq_kurt': 16.636522021084577}


49it [00:01, 33.01it/s]


...

{'method': 'tauleaping', 'g_steps': 0, 'tau_steps': 50, 'use_corrector': True, 'corrector': 'barker', 'c_stepsize': 1.0, 'c_steps': 2, 'nfe': 140.0, 'time_mean': 0.0017216196541281824, 'time_var': 0.000915663401181734, 'time_skew': 0.4254793345553095, 'time_kurt': 1.5797153602551395, 'freq_mean': 0.001721625158235898, 'freq_var': 0.001739948233790971, 'freq_skew': 0.9388885858353935, 'freq_kurt': 10.052204993236318}


49it [00:01, 33.01it/s]


{'method': 'tauleaping', 'g_steps': 0, 'tau_steps': 50, 'use_corrector': True, 'corrector': 'barker', 'c_stepsize': 1.0, 'c_steps': 2, 'nfe': 140.0, 'time_mean': 0.0031444225722150345, 'time_var': 0.0008217112031837417, 'time_skew': 0.44892493807601647, 'time_kurt': 1.6076522432212543, 'freq_mean': 0.003144427678805584, 'freq_var': 0.0019123694411548222, 'freq_skew': 1.0118940578347422, 'freq_kurt': 10.065675116100106}


21it [00:00, 36.77it/s]


KeyboardInterrupt: 

In [8]:
results_file = 'gene_results.csv'

num_repeats = 20
sample_size = 500

methods = ["tauleaping", 
           "gillespies"]

updates_per_evals = [3, 2, 1]
num_sample_steps = [20, 30, 60]

correctors = ["mpf", "barker", "birthdeath"]
corrector_stepsizes = [0.1, 0.1, 0.1]
corrector_steps = 1

# 1 corrector step
c_steps = 1
for method in methods:
    for i in range(len(correctors)):
        
        corrector = correctors[i]
        c_stepsize = corrector_stepsizes[i]
        
        for j in range(len(updates_per_evals)):
        
            g_steps = updates_per_evals[j]
            tau_steps = num_sample_steps[j]
            
            run_experiments(num_repeats=num_repeats, 
                            sample_size=sample_size, 
                            results_file=results_file,
                            method=method, 
                            g_steps=g_steps, 
                            tau_steps=tau_steps, 
                            c_steps=c_steps, 
                            c_stepsize=c_stepsize, 
                            corrector=corrector)

19it [00:01, 14.06it/s]


{'method': 'tauleaping', 'g_steps': 0, 'tau_steps': 20, 'use_corrector': True, 'corrector': 'mpf', 'c_stepsize': 0.1, 'c_steps': 1, 'nfe': 38.0, 'time_mean': 0.001453785830588084, 'time_var': 0.00011966448159442669, 'time_skew': 0.03543744400599668, 'time_kurt': 0.45229845978696254, 'freq_mean': 0.0014537864171571193, 'freq_var': 0.0014596635940522433, 'freq_skew': 0.3590237116976977, 'freq_kurt': 3.9415354976449333}


19it [00:01, 14.03it/s]


{'method': 'tauleaping', 'g_steps': 0, 'tau_steps': 20, 'use_corrector': True, 'corrector': 'mpf', 'c_stepsize': 0.1, 'c_steps': 1, 'nfe': 38.0, 'time_mean': 0.0011457062184635, 'time_var': 7.374185306289803e-05, 'time_skew': 0.02132352609229032, 'time_kurt': 0.11389081260768212, 'freq_mean': 0.0011457068471495354, 'freq_var': 0.001077168374591899, 'freq_skew': 0.38299093653066685, 'freq_kurt': 3.4920372551665433}


19it [00:01, 14.05it/s]


...

{'method': 'gillespies', 'g_steps': 1, 'tau_steps': 0, 'use_corrector': True, 'corrector': 'birthdeath', 'c_stepsize': 0.1, 'c_steps': 1, 'nfe': 114.0, 'time_mean': 0.0016438248259349243, 'time_var': 0.00012717981097407993, 'time_skew': 0.03507016014582662, 'time_kurt': 0.17856079572506364, 'freq_mean': 0.0016438244818586063, 'freq_var': 0.0020349561720031553, 'freq_skew': 0.5227694485739164, 'freq_kurt': 5.847608130216171}


100%|██████████| 60/60 [00:05<00:00, 11.84it/s]


{'method': 'gillespies', 'g_steps': 1, 'tau_steps': 0, 'use_corrector': True, 'corrector': 'birthdeath', 'c_stepsize': 0.1, 'c_steps': 1, 'nfe': 114.0, 'time_mean': 0.0010369684621620262, 'time_var': 0.0002832955408943039, 'time_skew': 0.028624876050235892, 'time_kurt': 0.17170936267936732, 'freq_mean': 0.0010369685503773861, 'freq_var': 0.0026853761322218886, 'freq_skew': 0.43528865067220773, 'freq_kurt': 5.076482798381343}


100%|██████████| 60/60 [00:04<00:00, 13.39it/s]

{'method': 'gillespies', 'g_steps': 1, 'tau_steps': 0, 'use_corrector': True, 'corrector': 'birthdeath', 'c_stepsize': 0.1, 'c_steps': 1, 'nfe': 114.0, 'time_mean': 0.0010537501595343354, 'time_var': 0.00025454114772700706, 'time_skew': 0.026819835114584703, 'time_kurt': 0.151735867358184, 'freq_mean': 0.0010537496827893393, 'freq_var': 0.00209119167016313, 'freq_skew': 0.5301547732105341, 'freq_kurt': 4.101824386122763}
Experiment results saved to  gene_results.csv


In [9]:
# 2 corrector steps
c_steps = 2
for method in methods:
    for i in range(len(correctors)):
        
        corrector = correctors[i]
        c_stepsize = corrector_stepsizes[i]
        
        for j in range(len(updates_per_evals)):
        
            g_steps = updates_per_evals[j]
            tau_steps = num_sample_steps[j]
            
            run_experiments(num_repeats=num_repeats, 
                            sample_size=sample_size, 
                            results_file=results_file,
                            method=method, 
                            g_steps=g_steps, 
                            tau_steps=tau_steps, 
                            c_steps=c_steps, 
                            c_stepsize=c_stepsize, 
                            corrector=corrector)

19it [00:01,  9.65it/s]


{'method': 'tauleaping', 'g_steps': 0, 'tau_steps': 20, 'use_corrector': True, 'corrector': 'mpf', 'c_stepsize': 0.1, 'c_steps': 2, 'nfe': 56.0, 'time_mean': 0.001009835578769906, 'time_var': 0.00011686038145943322, 'time_skew': 0.02917878574166684, 'time_kurt': 0.2544085870669534, 'freq_mean': 0.0010098358234793261, 'freq_var': 0.0010308633306742073, 'freq_skew': 0.3475818708638271, 'freq_kurt': 2.716082843609288}


19it [00:01,  9.65it/s]


{'method': 'tauleaping', 'g_steps': 0, 'tau_steps': 20, 'use_corrector': True, 'corrector': 'mpf', 'c_stepsize': 0.1, 'c_steps': 2, 'nfe': 56.0, 'time_mean': 0.0008104020288032581, 'time_var': 4.38077101274296e-05, 'time_skew': 0.022802815375744176, 'time_kurt': 0.15596773154138605, 'freq_mean': 0.000810402089226342, 'freq_var': 0.0007350986173563167, 'freq_skew': 0.2733815457264513, 'freq_kurt': 2.9165032073465604}


19it [00:01,  9.64it/s]


...

{'method': 'gillespies', 'g_steps': 1, 'tau_steps': 0, 'use_corrector': True, 'corrector': 'birthdeath', 'c_stepsize': 0.1, 'c_steps': 2, 'nfe': 168.0, 'time_mean': 0.0017070810382580075, 'time_var': 0.0004676773887696341, 'time_skew': 0.049819765184278456, 'time_kurt': 0.33598621690011615, 'freq_mean': 0.0017070803982971122, 'freq_var': 0.004450299991383365, 'freq_skew': 0.6274016255778111, 'freq_kurt': 5.500371621048389}


100%|██████████| 60/60 [00:06<00:00,  8.86it/s]


{'method': 'gillespies', 'g_steps': 1, 'tau_steps': 0, 'use_corrector': True, 'corrector': 'birthdeath', 'c_stepsize': 0.1, 'c_steps': 2, 'nfe': 168.0, 'time_mean': 0.0009765296342553823, 'time_var': 0.00036106526696234393, 'time_skew': 0.029477624724209908, 'time_kurt': 0.1882193971792337, 'freq_mean': 0.0009765297618103871, 'freq_var': 0.0029105925447813477, 'freq_skew': 0.4696419420891681, 'freq_kurt': 4.4861022819258185}


100%|██████████| 60/60 [00:06<00:00,  9.10it/s]

{'method': 'gillespies', 'g_steps': 1, 'tau_steps': 0, 'use_corrector': True, 'corrector': 'birthdeath', 'c_stepsize': 0.1, 'c_steps': 2, 'nfe': 168.0, 'time_mean': 0.0012482264021589772, 'time_var': 0.0001668604152618891, 'time_skew': 0.030627325814674845, 'time_kurt': 0.1894970795362538, 'freq_mean': 0.0012482259847038516, 'freq_var': 0.0012384891904299547, 'freq_skew': 0.40527461180609053, 'freq_kurt': 3.470890517211541}
Experiment results saved to  gene_results.csv
